In [1]:
# Parameters
kms_key = "arn:aws:kms:us-west-2:000000000000:1234abcd-12ab-34cd-56ef-1234567890ab"


# SKLearn Script Mode + Bring Your Own Model

- [Documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/using_sklearn.html)
- Dataset: [Iris](https://archive.ics.uci.edu/ml/datasets/iris)

## Read Data

In [2]:
import boto3
import pandas as pd
import numpy as np

s3 = boto3.client("s3")
s3.download_file(f"sagemaker-sample-files", "datasets/tabular/iris/iris.data", "iris.data")

df = pd.read_csv(
    "iris.data", header=None, names=["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"]
)
df.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# Convert the three classes from strings to integers in {0,1,2}
df["class_cat"] = df["class"].astype("category").cat.codes
categories_map = dict(enumerate(df["class"].astype("category").cat.categories))
print(categories_map)
df.head()

{0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}


,sepal_len,sepal_wid,petal_len,petal_wid,class,class_cat
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0


In [4]:
# Split the data into 80-20 train-test split
num_samples = df.shape[0]
split = round(num_samples * 0.8)
train = df.iloc[:split, :]
test = df.iloc[split:, :]
print("{} train, {} test".format(split, num_samples - split))

120 train, 30 test


In [5]:
# Write train and test CSV files
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

## Upload Data to S3

In [6]:
# Create a sagemaker session to upload data to S3
import sagemaker

sagemaker_session = sagemaker.Session()

# Upload data to default S3 bucket
prefix = "DEMO-sklearn-iris"
training_input_path = sagemaker_session.upload_data("train.csv", key_prefix=prefix + "/training")

## Train Estimator

In [7]:
# Use the current execution role for training. It needs access to S3
role = sagemaker.get_execution_role()
print(role)

Couldn't call 'get_role' to get Role ARN from role name SageMakerRole to get Role path.


arn:aws:iam::000000000000:role/SageMakerRole


In [8]:
# Docs: https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html

from sagemaker.sklearn import SKLearn

sk_estimator = SKLearn(
    entry_point="train.py",
    role=role,
    instance_count=1,
    instance_type="ml.c5.xlarge",
    py_version="py3",
    framework_version="0.23-1",
    script_mode=True,
    hyperparameters={"estimators": 20},
)

# Train the estimator
sk_estimator.fit({"train": training_input_path})

2022-03-23 18:04:38 Starting - Starting the training job.

.

.


2022-03-23 18:05:05 Starting - Preparing the instances for trainingProfilerReport-1648058678: InProgress
.

.

...

.


2022-03-23 18:07:06 Training - Training image download completed. Training in progress..

2022-03-23 18:07:05,310 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-03-23 18:07:05,315 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-23 18:07:05,324 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-03-23 18:07:05,623 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-23 18:07:05,642 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-23 18:07:05,656 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-03-23 18:07:05,667 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_sklearn_container.training:main",
    "hosts": [
        "algo-1"
    ],
    


2022-03-23 18:07:26 Uploading - Uploading generated training model
2022-03-23 18:07:26 Completed - Training job completed


Training seconds: 77
Billable seconds: 77


## Deploy Endpoint

In [9]:
import time

sk_endpoint_name = "sklearn-rf-model" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sk_predictor = sk_estimator.deploy(
    initial_instance_count=1, instance_type="ml.m5.large", endpoint_name=sk_endpoint_name
)

-

-

-

-

-

-

!

## Test Endpoint
- Can use [invoke endpoint](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html) or [predictor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-predictor), using invoke endpoint for this example. 
- For predictor make sure to [serialize](https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html) properly.

In [10]:
import json

client = sagemaker_session.sagemaker_runtime_client

request_body = {"Input": [[9.0, 3571, 1976, 0.525]]}
data = json.loads(json.dumps(request_body))
payload = json.dumps(data)

response = client.invoke_endpoint(
    EndpointName=sk_endpoint_name, ContentType="application/json", Body=payload
)

result = json.loads(response["Body"].read().decode())["Output"]
print("Predicted class category {} ({})".format(result, categories_map[result]))

Predicted class category 1 (Iris-versicolor)


## Cleanup

In [11]:
sk_predictor.delete_model()
sk_predictor.delete_endpoint()